### General Set Up

In [23]:
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import requests


In [24]:
## Creating XML Tree Object ##

TARGET = 'servicestatus.json'
SOURCE = 'http://web.mta.info/status/serviceStatus.txt'

response = requests.get(SOURCE)
xml_string = response.text
root = ET.fromstring(xml_string)
# ET.fromstring converts a string of XML content to a XML tree object.


### Function to intake xml root and return full list of MTA lines present in the XML. 
(Should always be the same.) 

In [27]:
def getFullLineList(xmlroot):
    FullLineList = []
    num = 0
    
    while num < len(root[2]):
        FullLineList.append((num,root[2][num][0].text))
        num += 1
        
    return FullLineList

FullLineList = getFullLineList(root)
FullLineList

[(0, '123'),
 (1, '456'),
 (2, '7'),
 (3, 'ACE'),
 (4, 'BDFM'),
 (5, 'G'),
 (6, 'JZ'),
 (7, 'L'),
 (8, 'NQR'),
 (9, 'S'),
 (10, 'SIR')]

### Function to intake service status xml object and return lines with service changes as a list of tuples

In [28]:
  
def Return_SC_Lines(xmlroot):
    # xmlroot must be an xml object of the MTA Service Status page using xml Element Tree package
    num_lines = list(range(0,len(xmlroot[2])))
    SC_lines = []
    
    for ea in num_lines:
        if xmlroot[2][ea][1].text != 'GOOD SERVICE':
            SC_lines.append((xmlroot[2][ea][0].text,root[2][ea][1].text))
        else:
            pass 
        
    #SC_Lines is a list of tuples with the line name and service status 
    return SC_lines
    
SC_lines = Return_SC_Lines(root)

### Function to take line name and service status XML and return beautiful soup of that line's service status 

In [29]:
def MakeLine_ServiceStatusSoup(line_name,xmlroot): 
    line_iq = line_name
    line_iq_num = ''
    
    for ea in FullLineList:
        if ea[1] == line_iq:
            line_iq_num = ea[0]
    
    # get string and make soup from line in question's status
    status_str = root[2][line_iq_num][2].text
    linestatus_soup = BeautifulSoup(status_str, 'lxml')
    
    return linestatus_soup


# bdfmsoup = MakeLine_ServiceStatusSoup('BDFM',root)
    
    
    

### Function intake service status xml and a line with planned work and return description of the planned work

In [33]:
def plannedWork_Simple(line_servicestatus_soup):
    
    tag_options = ['span']
    classname_options = ['TitlePlannedWork']
    subtag_options = ['p']
    pwSimple_text = ''

    for spans in line_servicestatus_soup.find_all(tag_options, {'class':classname_options}):
        subs = spans.find_all_next(subtag_options)
        for each in subs:
            pwSimple_text += each.text
            
    return pwSimple_text


# this works for printing summary line, and not when planned work is a span

def plannedWork_Detail(line_servicestatus_soup):
    
    tag_options = ['a']
    classname_options = ['plannedWorkDetailLink']
    subtag_options = ['div','p']
    pwDetail_text = ''
    
    for a_summ in line_servicestatus_soup.find_all(tag_options, {'class':classname_options}):
        pw_summtext = a_summ.text

        #print(pw_summtext,'\n', next_div, '\n')
        pwDetail_text += pw_summtext + ' '
    
    return pwDetail_text
        


def PlannedWorkText(line_name,xmlroot):
    
    line_status_soup = MakeLine_ServiceStatusSoup(line_name,xmlroot)
    service_string = ''
    
    line_iq = line_name
    line_iq_num = ''
    
    for ea in FullLineList:
        if ea[1] == line_iq:
            line_iq_num = ea[0]
        
        
    if len(line_status_soup.find_all('a', {'class':'plannedWorkDetailLink'})) < 1:
        service_string = plannedWork_Simple(line_status_soup)
            
    else:
        service_string = plannedWork_Detail(line_status_soup)

        
        
    return [line_name, xmlroot[2][line_iq_num][1].text, service_string]
    
    
print(PlannedWorkText('123',root)[2])
 

TRACK MAINTENANCE[3] Downtown trains run local from 96 St to Times Sq-42 St TRACK MAINTENANCE[2] Downtown trains run local from 96 St to Times Sq-42 St TUNNEL RECONSTRUCTION  Weekend [2] [3] station closures and route changes TRACK MAINTENANCE[1] South Ferry-bound trains skip 125, 116, 110 and 103 Sts TRACK REPLACEMENT[1] Van Cortlandt Park-bound trains skip 50 St, 59 St and 66 St 


### Function to intake service status xml and a line name and return text of delays on that line, if applicable 

In [22]:
def delays_text(line_name,xmlroot):

    line_status_soup = MakeLine_ServiceStatusSoup(line_name,xmlroot)
    delay_string = ''
    
    delayline_name = line_name
    dealyline_num = ''
    
    for ea in FullLineList:
        if ea[1] == delayline_name:
            dealyline_num = ea[0]


    del_text = ''
    
    for del1 in line_status_soup.find_all('span', {'class': 'TitleDelay'}):
        
        # below checks that the delay description is contained in <p> tags
        if len(del1.find_all('p')) > 0:
            
            delay_deets = del1.find_all('p')
            
            for dels in delay_deets:
                del_text += dels.text + ' '
            
            
        else:
            del_text = ''.join(line_status_soup.find_all(text=True)[3:5]).strip()
            
            
    return del_text
            
            

print(delays_text('NQR',root))


[R] train service has resumed following an earlier incident involving debris on the tracks at 86 St.
